## Build Functions for ELT

In [1]:
# Install Requirments (Updated on 10/2/2024, streamlit 1.39.0)
# !pip3 install -r requirements.txt

In [2]:
import pandas as pd
import numpy as np
from IPython.display import display
from yfinance import Ticker
from pykalman import KalmanFilter

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer

from sklearn.pipeline import Pipeline

from xgboost import XGBClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from lightgbm import LGBMClassifier
from sklearn.neighbors import KNeighborsClassifier

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report

from sklearn.model_selection import cross_val_predict

from datetime import datetime
import pytz

import pprint

from src import modules as f # f is for function

%load_ext autoreload
%autoreload 2

### Download, Transform, and Modeling All in One

In [3]:
# manufactoring suppliers 
# 'lrcs', 'klac', ''

In [10]:
f.predictions('dell')

DELL 5m Interval Timestamp: 2024-11-25 03:24:51 


,XGBoost,GradientBoosting,RandomForest,KNN
prediction,static,static,up,static
kelly_1:2.5,0.215663,0.210256,0.241096,0.117117
prob_up,0.353431,0.351316,0.391594,0.25
prob_static,0.443547,0.44872,0.365148,0.5
prob_down,0.203022,0.199964,0.243258,0.25
precision,0.439759,0.435897,0.457926,0.369369
recall,0.454357,0.458506,0.482474,0.510373
f1,0.446939,0.446916,0.46988,0.428571
support,"[485.0, 482.0, 484.0]","[485.0, 482.0, 484.0]","[485.0, 482.0, 484.0]","[485.0, 482.0, 484.0]"


DELL 15m Interval Timestamp: 2024-11-25 03:24:58 


,XGBoost,GradientBoosting,RandomForest,KNN
prediction,static,static,static,static
kelly_1:2.5,0.205618,0.136527,0.134831,0.082759
prob_up,0.12245,0.267318,0.244097,0.125
prob_static,0.689679,0.438963,0.440403,0.5
prob_down,0.187871,0.293719,0.315499,0.375
precision,0.432584,0.383234,0.382022,0.344828
recall,0.472393,0.392638,0.417178,0.490798
f1,0.451613,0.387879,0.398827,0.405063
support,"[162.0, 163.0, 161.0]","[162.0, 163.0, 161.0]","[162.0, 163.0, 161.0]","[162.0, 163.0, 161.0]"


DELL 1h Interval Timestamp: 2024-11-25 03:25:01 


...

,XGBoost,GradientBoosting,RandomForest,KNN
prediction,down,down,static,up
kelly_1:2.5,0.144444,0.175,0.07972,0.136842
prob_up,0.288912,0.293651,0.306432,0.375
prob_static,0.351927,0.332856,0.375963,0.25
prob_down,0.359161,0.373492,0.317605,0.375
precision,0.388889,0.410714,0.342657,0.383459
recall,0.357664,0.335766,0.360294,0.372263
f1,0.372624,0.369478,0.351254,0.377778
support,"[137.0, 136.0, 137.0]","[137.0, 136.0, 137.0]","[137.0, 136.0, 137.0]","[137.0, 136.0, 137.0]"


DELL 1wk Interval Timestamp: 2024-11-25 03:25:11 


,XGBoost,GradientBoosting,RandomForest,KNN
prediction,static,static,static,static
kelly_1:2.5,0.0,0.096774,0.2125,0.217647
prob_up,0.344585,0.259923,0.286786,0.125
prob_static,0.569762,0.630705,0.471521,0.5
prob_down,0.085652,0.109372,0.241693,0.375
precision,0.285714,0.354839,0.4375,0.441176
recall,0.230769,0.423077,0.269231,0.576923
f1,0.255319,0.385965,0.333333,0.5
support,"[27.0, 26.0, 28.0]","[27.0, 26.0, 28.0]","[27.0, 26.0, 28.0]","[27.0, 26.0, 28.0]"


DELL 1mo Interval Timestamp: 2024-11-25 03:25:13 


,XGBoost,GradientBoosting,RandomForest,KNN
prediction,down,down,down,down
kelly_1:2.5,-0.05,0.3,0.3,-0.05
prob_up,0.008164,0.004356,0.114231,0.125
prob_static,0.102976,0.191951,0.347402,0.25
prob_down,0.888861,0.803693,0.538366,0.625
precision,0.25,0.5,0.5,0.25
recall,0.2,0.4,0.6,0.2
f1,0.222222,0.444444,0.545455,0.222222
support,"[5.0, 4.0, 5.0]","[5.0, 4.0, 5.0]","[5.0, 4.0, 5.0]","[5.0, 4.0, 5.0]"


In [ ]:
symbol='NVDA'
interval='1d'

# # Define Eastern Time Zone
# eastern = pytz.timezone('US/Eastern')

# # Get current time in Eastern Time Zone
# eastern_time = datetime.now(eastern)

# # Format the time to include hour, minute, and seconds
# time_stamp = eastern_time.strftime('%Y-%m-%d %H:%M:%S')

# print(f'DL Time: {time_stamp}')

# f.download(symbol, interval, period)
f.transform(symbol, interval)
curr_prediction, models, feature_names, classification_reports = f.model(symbol, interval)
predictions, prediction_probas = f.make_prediction(models, curr_prediction, feature_names)

f.predictions_summary(predictions, prediction_probas, classification_reports)

In [ ]:
models

In [ ]:
models['XGBoost'].feature_importances_

In [ ]:
feature_names

In [ ]:
merged_list = sorted(zip(feature_names, models['XGBoost'].feature_importances_),
                     key=lambda x: x[1], 
                     reverse=True
                    )
merged_list

In [ ]:
predictions

In [ ]:
classification_reports

### Hyperparameter Searches

In [ ]:
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, train_test_split
from sklearn.metrics import classification_report
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import pandas as pd
import numpy as np

def model(symbol, interval, search_type='none'):
    # Load data
    data = f.load_model_df(symbol, interval)
    data.dropna(inplace=True, axis=0)
    X = data.drop(columns=['direction'], axis=1)
    y = data['direction']
    
    # Print column names to check for issues
    print("Columns in X before preprocessing:")
    print(X.columns)
    
    # Remove duplicate columns
    X = X.loc[:, ~X.columns.duplicated()]
    
    # Check if categorical_features are present in X
    categorical_features = ['day_of_month', 'day_of_week', 'hour_of_day']
    missing_features = [col for col in categorical_features if col not in X.columns]
    if missing_features:
        print(f"Missing categorical features: {missing_features}")
    
    # Store current prediction data (last row)
    curr_prediction = X.iloc[-1].copy()

    # Drop last row from X and y to prevent the model from seeing it
    X = X.iloc[:-1]
    y = y.iloc[:-1]
    
    # Create the categorical transformer
    categorical_transformer = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
    
    # Create the preprocessor
    preprocessor = ColumnTransformer(
        transformers=[
            ('cat', categorical_transformer, categorical_features)
        ],
        force_int_remainder_cols=False # This will include all other columns in the transformed output
    )
    
    # Define your models
    models = {
        'XGBoost': XGBClassifier(random_state=42, n_jobs=-1),
        'RandomForest': RandomForestClassifier(random_state=42, n_jobs=-1),
        'GradientBoosting': GradientBoostingClassifier(random_state=42, validation_fraction=0.25, n_iter_no_change=31),
        # 'LightGBM': LGBMClassifier(random_state=42,force_col_wise=True),
        'KNN': KNeighborsClassifier(n_neighbors=7, p=1,weights='distance')
    }
    
    # Hyperparameters to search
    param_grids = {
        'XGBoost': {
            'classifier__n_estimators': [100, 200, 300],
            'classifier__max_depth': [3, 5, 7, 9],
            'classifier__learning_rate': [0.01, 0.1, 0.2, 0.3]
        },
        'RandomForest': {
            'classifier__n_estimators': [100, 200, 300],
            'classifier__max_depth': [None, 10, 20, 30],
            'classifier__min_samples_split': [2, 5, 10, 13]
        },
        'GradientBoosting': {
            'classifier__n_estimators': [100, 200, 300, 400],
            'classifier__max_depth': [3, 5, 7, 13],
            'classifier__le||arning_rate': [0.01, 0.1, 0.2, 0.5]
        },
        'KNN': {
            'classifier__n_neighbors': [3, 5, 7, 13],
            'classifier__weights': ['uniform', 'distance'],
            'classifier__p': [1, 2]  # 1: Manhattan, 2: Euclidean
        }
    }
    
    # Create a function to get feature names after transformation
    def get_feature_names_out(column_transformer):
        feature_names = []
        for name, transformer, columns in column_transformer.transformers_:
            if transformer == 'drop' or transformer == 'passthrough':
                if transformer == 'passthrough':
                    feature_names.extend(columns)
                continue
            if hasattr(transformer, 'get_feature_names_out'):
                names = transformer.get_feature_names_out(columns)
                feature_names.extend(names)
            else:
                feature_names.extend(columns)
        return feature_names
    
    # Split data before preprocessing to avoid data leakage
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, stratify=y, test_size=0.2, random_state=42)
    
    # Fit the preprocessor on training data
    preprocessor.fit(X_train)
    
    # Transform training and test data
    X_train_transformed = preprocessor.transform(X_train)
    X_test_transformed = preprocessor.transform(X_test)
    
    # Get feature names after transformation
    feature_names = get_feature_names_out(preprocessor)
    
    # Convert transformed data to DataFrame
    X_train_transformed = pd.DataFrame(X_train_transformed, columns=feature_names)
    X_test_transformed = pd.DataFrame(X_test_transformed, columns=feature_names)
    
    # Transform curr_prediction
    curr_prediction_transformed = preprocessor.transform(
        curr_prediction.to_frame().T)
    curr_prediction_transformed = pd.DataFrame(
        curr_prediction_transformed, columns=feature_names)
    
    for model_name, model in models.items():
        # Create a pipeline with the classifier
        pipeline = Pipeline(steps=[
            ('classifier', model)
        ])
        
        # Get the parameter grid for the current model
        param_grid = param_grids.get(model_name, {})
        
        # Use GridSearchCV or RandomizedSearchCV
        if search_type == 'grid' and param_grid:
            search = GridSearchCV(
                pipeline, param_grid, cv=5, scoring='accuracy', n_jobs=-1)
        elif search_type == 'random' and param_grid:
            search = RandomizedSearchCV(
                pipeline, param_grid, cv=5, scoring='accuracy',
                n_jobs=-1, n_iter=10, random_state=42)
        else:
            search = pipeline
        
        # Fit the model
        search.fit(X_train_transformed, y_train)
        
        # If using search, get the best estimator
        if search_type in ['grid', 'random'] and param_grid:
            best_model = search.best_estimator_
            print(f"Best parameters for {model_name}: {search.best_params_}")
            model = best_model.named_steps['classifier']
        else:
            model = search.named_steps['classifier']
        
        # Store the model
        models[model_name] = model
        
        # Predict on test data
        y_pred = search.predict(X_test_transformed)
        
        # Evaluate the model
        print(f"Model: {model_name}")
        print(classification_report(y_test, y_pred, zero_division=0))
    
    return curr_prediction_transformed, models, feature_names

In [ ]:
curr_prediction, models, feature_names = model('AMD', '5m', 'grid')